In [1]:
import nrrd

In [2]:
# sample image
im_path = 'data/images/LIDC-IDRI-0001.nrrd'

im_V,im_d = nrrd.read(im_path)

In [3]:
im_V.shape

(512, 512, 133)

In [4]:
im_d

OrderedDict([('type', 'int16'),
             ('dimension', 3),
             ('sizes', array([512, 512, 133])),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('patient_id', 'LIDC-IDRI-0001'),
             ('slice_thickness', '2.5'),
             ('pixel_spacing', '0.703125'),
             ('contrast_used', 'True')])

In [17]:
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv
from scipy.ndimage import rotate   # for visualization purposes 

In [6]:
isv(im_V)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [7]:
# sample image
msk_path = 'data/masks/LIDC-IDRI-0001.nrrd'

msk_V,msk_d = nrrd.read(msk_path)

isv(msk_V)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [19]:
from skimage.measure import label,regionprops

# label the mask into connected regions
nodule_labels,num_labels = label(msk_V,return_num=True)

In [39]:
nodule_props = regionprops(nodule_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(len(nodule_props)))        

There are 1 nodule(s) in this image.


In [47]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
nodule_props[0].centroid
centroid = [round(i) for i in nodule_props[0].centroid]
centroid

[367, 316, 90]

In [26]:
from skimage.segmentation import watershed

watershed?

In [56]:
import numpy as np

marker_image = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=np.uint8)
marker_image[centroid] = 1
marker_image[300,350,90] = 2

In [57]:
result = watershed(im_V,marker_image,watershed_line=True)

In [58]:
result.shape
isv(result)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [42]:
# label the mask into connected regions
result_labels,num_results = label(result,return_num=True)

result_props = regionprops(result_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(len(result_props)))   

There are 1 nodule(s) in this image.


In [43]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
result_props[0].centroid
[round(i) for i in nodule_props[0].centroid]

[367, 316, 90]

In [63]:
from skimage.morphology import ball
from scipy import ndimage as ndi
from skimage.filters import rank
from scipy.ndimage import median_filter as median

# denoise image
denoised = median(im_V,5)

print(denoised.shape)
isv(denoised)

(512, 512, 133)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [66]:
# # find continuous region (low gradient -
# # where less than 10 for this image) --> markers
# # disk(5) is used here to get a more smooth image
# markers = rank.gradient(denoised, ball(5)) < 10
# markers = ndi.label(markers)[0]

# # local gradient (disk(2) is used to keep edges thin)
# gradient = rank.gradient(denoised, ball(2))

# # process the watershed
# labels = watershed(gradient, markers)

ValueError: The parameter `image` must be a 2-dimensional array

In [72]:
from skimage.morphology import erosion, dilation, ball

# try some funky stuff
max_image = dilation(denoised,ball(2))
min_image = erosion(denoised,ball(2))

gradient_image = max_image - min_image

isv(gradient_image)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [84]:
from skimage.measure import label

threshold_gradient = gradient_image < 300
isv(threshold_gradient)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [85]:
markers = label(threshold_gradient)
isv(markers)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [86]:
# process the watershed
labels = watershed(gradient_image, markers)

In [81]:
isv(labels)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…